In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.metrics import average_precision_score
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM

In [2]:
DATASET = "DatasetUpdate/MMS (1).csv"
TOPOLOGY = "DatasetUpdate/MMS_topology.pk"

data = pd.read_csv(DATASET, header=[0,1])

labels = data['label']
metric = data.drop(['TimeStamp', 'label'], axis = 1)
metric.columns.names = ['pod','metric']
tempm = metric.swaplevel('metric','pod',axis=1).stack()

tempm = (tempm-tempm.mean())/(tempm.std())
metric = tempm.unstack().swaplevel('metric','pod',axis=1).stack().unstack()

with open(TOPOLOGY, 'rb') as f:
    edge_index = pickle.load(f)

/opt/anaconda/lib/python3.7/site-packages/pandas/core/generic.py:3889: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


# 填充缺失值

In [3]:
metric.interpolate(inplace=True)
metric.bfill(inplace=True)

# 划分数据集合

In [4]:
data_scaled = metric.values

def train_test_split(data,labels, train_portion):
    time_len = data.shape[0]
    train_size = int(time_len * train_portion)
    train_data = np.array(data[:train_size,...])
    test_data = np.array(data[train_size:,...])
    
    train_y = np.array(labels[:train_size])
    test_y = np.array(labels[train_size:])
    return train_data, test_data, train_y, test_y

train_rate = 0.67
train_data, test_data, train_y, test_y = train_test_split(data_scaled,labels,train_rate)

# 对比实验： LOF

In [5]:
clf = LocalOutlierFactor(novelty=True)
clf.fit(train_data)
pred = - clf.score_samples(test_data)
labels = test_y.flatten()
ap = average_precision_score(labels,pred)
print("LOF的AP得分为:",ap)

LOF的AP得分为: 0.4367750588451614


# 对比实验： OCSVM

In [6]:
clf = OneClassSVM()
clf.fit(train_data)
pred = - clf.score_samples(test_data)
labels = test_y.flatten()
ap = average_precision_score(labels,pred)
print("SVM的AP得分为:",ap)

SVM的AP得分为: 0.18937963664738064
